In [ ]:
# %%
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# %%
with open("./yolo/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# %%
classes

# %%
len(classes)

# %%
# Load YOLO net
net = cv2.dnn.readNet("./yolo/yolov3.weights", "./yolo/yolov3.cfg")

# %%
# Get layers names
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# %%
get_output_layers(net)

# %%
def draw_bounding_boxes(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = (255, 0, 0) # color = [float(c) for c in np.random.uniform(0, 255, size=(3,))]
    cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(img, label, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# %%
image = cv2.imread("../02_interpolation_feature_extraction/images/horse.jpg")

# %%
width, height = image.shape[1], image.shape[0]
scale = 0.00392

# %%
blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(get_output_layers(net))

# %%
blob.shape

# %%
outs[0].shape

# %%
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > conf_threshold:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

# Remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# %%
for i in indices:
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    draw_bounding_boxes(image, class_ids[i], confidences[i], round(x), round(y), round(x + w), round(y + h))

# %%
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 8))
plt.imshow(rgb_image)
plt.axis('off')
plt.show()



In [ ]:
# %%
def detect_objects_in_image(image_path):
    image = cv2.imread(image_path)

    width, height = image.shape[1], image.shape[0]
    scale = 0.00392

    blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Remove overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]
        draw_bounding_boxes(image, class_ids[i], confidences[i], round(x), round(y), round(x + w), round(y + h))

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 8))
    plt.imshow(rgb_image)
    plt.axis('off')
    plt.show()

# %%
detect_objects_in_image("./sample.jpg")



In [ ]:

# %%
detect_objects_in_image("./sample2.jpg")